# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298645698923                   -0.85    5.0
  2   -8.300224547053       -2.80       -1.25    1.1
  3   -8.300439046154       -3.67       -1.89    1.2
  4   -8.300461563888       -4.65       -2.74    3.8
  5   -8.300464389019       -5.55       -3.20    2.5
  6   -8.300464508188       -6.92       -3.35   10.9
  7   -8.300464573674       -7.18       -3.50    1.5
  8   -8.300464610463       -7.43       -3.64    2.5
  9   -8.300464633587       -7.64       -3.82    1.4
 10   -8.300464639645       -8.22       -3.96    3.8
 11   -8.300464643759       -8.39       -4.31    1.9


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63706390919                   -0.71    5.9
  2   -16.67780199772       -1.39       -1.14    1.4
  3   -16.67887052246       -2.97       -1.86    3.5
  4   -16.67917396788       -3.52       -2.48    2.8
  5   -16.67927029008       -4.02       -3.03    2.6
  6   -16.67928617095       -4.80       -3.51    4.4
  7   -16.67928619421       -7.63       -3.85    2.0
  8   -16.67928621984       -7.59       -4.48    1.4


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32554445882                   -0.56    6.8
  2   -33.33268669821       -2.15       -1.00    1.2
  3   -33.33413196105       -2.84       -1.73    4.8
  4   -33.33428210757       -3.82       -2.64    2.5
  5   -33.33694085453       -2.58       -2.50    4.8
  6   -33.33694163087       -6.11       -2.54    8.2
  7   -33.33694385180       -5.65       -3.07    1.0
  8   -33.33694385341       -8.79       -3.53    3.9
  9   -33.33694392252       -7.16       -3.95    3.1
 10   -33.33694392642       -8.41       -4.42    4.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298074756319                   -0.85    5.0
  2   -8.300254122095       -2.66       -1.58    1.0
  3   -8.300413453472       -3.80       -2.43    3.2
  4   -8.300339072421   +   -4.13       -2.20    3.6
  5   -8.300464099733       -3.90       -3.50    1.0
  6   -8.300464538637       -6.36       -3.75    7.9
  7   -8.300464633261       -7.02       -4.11    1.0


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32668206255                   -0.56    7.0
  2   -33.30767679332   +   -1.72       -1.26    1.5
  3   -7.308813875588   +    1.41       -0.36    6.8
  4   -33.32415924169        1.42       -1.69    5.6
  5   -33.16882615675   +   -0.81       -1.29    3.9
  6   -30.41025631896   +    0.44       -0.83    4.1
  7   -33.32819136742        0.47       -1.99    4.4
  8   -33.33385658015       -2.25       -2.17    2.0
  9   -33.33598110803       -2.67       -2.33    2.4
 10   -33.33611724146       -3.87       -2.59    1.9
 11   -33.33673189131       -3.21       -2.86    2.0
 12   -33.33689141552       -3.80       -3.16    2.9
 13   -33.33694166424       -4.30       -3.47    3.5
 14   -33.33693973530   +   -5.71       -3.66    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.